In [131]:
import numpy as np
import random 

In [133]:
import numpy as np
import random # Importação necessária para o backtracking

class Sudoku:
    def __init__(self, ):
        self.matrix = np.zeros((9,9), dtype=np.int8)
        self.solution_matrix = np.zeros((9,9), dtype=np.int8) # Armazena o tabuleiro resolvido
        self.list_numbers = list(range(1,10))
        self.end = False
        self.dificult = 'easy'
        # Máscara para rastrear quais células o usuário pode modificar (True = editável)
        self.editable_mask = np.full((9,9), True, dtype=bool)
        self.erros = 0

    def get_invalid_values(self, n_row, n_col):
        invalid_values = list()

        # valores da linha
        invalid_values.extend(np.unique(self.matrix[n_row, :]).tolist())

        # valores da coluna
        invalid_values.extend(np.unique(self.matrix[:, n_col]).tolist())

        # valores do bloco 3x3
        start_row = (n_row // 3) * 3
        start_col = (n_col // 3) * 3
        block = self.matrix[start_row:start_row+3, start_col:start_col+3]
        invalid_values.extend(np.unique(block).tolist())

        # Remove o 0 se estiver presente, pois ele é um valor 'inválido' apenas por ser um espaço vazio
        row_col_section_invalid = set(invalid_values) - {0} 
        return row_col_section_invalid

    def level(self,):
        # Aumentei um pouco a dificuldade, ajustando o percentual de *números a manter*
        if self.dificult == 'easy':
            self.luck_percent = .50 # 50% dos números são mantidos
            self.max_err = 10
        elif self.dificult == 'medium':
            self.luck_percent = .35 # 35% dos números são mantidos
            self.max_err = 5
        else: 
            self.luck_percent = .20 # 20% dos números são mantidos
            self.max_err = 3

    def solve_matrix(self):
        """
        Método de backtracking para preencher o tabuleiro do Sudoku.
        """
        for i in range(9):
            for j in range(9):
                # Encontra a primeira célula vazia (valor 0)
                if self.matrix[i, j] == 0:
                    
                    # Tenta números de 1 a 9 em ordem aleatória para gerar tabuleiros diferentes
                    shuffled_numbers = random.sample(self.list_numbers, 9)
                    
                    for num in shuffled_numbers:
                        # Verifica se o número é válido para a posição (i, j)
                        if num not in self.get_invalid_values(i, j):
                            self.matrix[i, j] = num
                            
                            # Chamada recursiva: Se a chamada for bem-sucedida, retorna True
                            if self.solve_matrix():
                                return True
                            
                            # Se a chamada recursiva falhar, desfaz e tenta o próximo (backtrack)
                            self.matrix[i, j] = 0  
                    
                    # Se todos os números falharem, retorna False (o algoritmo retrocede)
                    return False
        
        # Se nenhuma célula vazia foi encontrada, o tabuleiro está preenchido e é válido
        return True

    def fill_shadow(self):
        """
        Cria o tabuleiro do jogo ao remover células do tabuleiro resolvido.
        """
        # Salva a matriz resolvida
        self.solution_matrix = self.matrix.copy()
        
        # Remove números da matriz principal para criar o jogo
        for i in range(9):
            for j in range(9):
                # Se o número for removido (com base na porcentagem de sorte/dificuldade)
                if random.random() > self.luck_percent: 
                    self.matrix[i, j] = 0
                    self.editable_mask[i, j] = True # A célula se torna editável
                else:
                    # Número permanece, não pode ser alterado
                    self.editable_mask[i, j] = False 

    def show_matrix(self,):
        # Adaptado para melhor visualização e para mostrar números fixos vs. jogados
        cont_row, cont_col = 0, 0
        print("\n  1 2 3   4 5 6   7 8 9")
        print("-------------------------")
        for i, row in enumerate(self.matrix):
            print('|', end=' ')
            for j, cel in enumerate(row):
                # Usa um caractere diferente para células editáveis vazias
                display_char = str(cel) if cel != 0 else '.'
                
                # Se a célula não for editável (era um número inicial), usa cor (se o terminal suportar)
                if not self.editable_mask[i, j] and cel != 0:
                    # \033[92m cor verde (para números iniciais) \033[0m reseta cor
                    print(f'\033[92m{display_char}\033[0m', end=' ') 
                else:
                    # Números que o usuário pode colocar ou já colocou
                    print(display_char, end=' ')
                
                cont_col += 1
                if cont_col == 3 and j == 8:
                    cont_col = 0
                    print(f'| {i+1}', end='') # Mostra o número da linha (1 a 9)

                elif cont_col == 3:
                    cont_col = 0
                    print('|', end=' ')

            cont_row += 1
            if cont_row == 3:
                cont_row = 0
                print('\n-------------------------')
            else:
                print('')
    

    def run(self):
        print('=== Bem-vindo ao jogo de Sudoku! ===')
        
        # 1. Configura a dificuldade
        valid_difficulties = ['easy', 'medium', 'hard']
        self.dificult = ''
        while self.dificult not in valid_difficulties:
            self.dificult = input('Escolha sua dificuldade ("easy", "medium" ou "hard"): ').lower()
            if self.dificult not in valid_difficulties:
                print("Dificuldade inválida. Tente novamente.")
                
        self.level()
        
        # 2. Gera o tabuleiro completo (e salva a solução)
        # Primeiro, zeramos o tabuleiro caso tenha sido jogado antes
        self.matrix = np.zeros((9,9), dtype=np.int8) 
        self.solve_matrix() # Popula o tabuleiro com a solução
        
        # 3. Cria o tabuleiro do jogo (oculta números)
        self.fill_shadow()
        
        while not self.end:
            self.show_matrix()
            
            # --- Entrada do Usuário ---
            try:
                print("\nPara jogar, insira (linha coluna valor), ex: 1 2 5")
                user_input = input("Ou digite 'sair' para encerrar: ").lower().split()
                
                if user_input[0] == 'sair':
                    self.end = True
                    print("Jogo encerrado.")
                    break
                
                if len(user_input) != 3:
                    print("Formato inválido. Use: linha coluna valor.")
                    continue
                    
                row, col, value = map(int, user_input)
                
                # Ajuste de 1-base (usuário) para 0-base (matriz)
                row -= 1
                col -= 1
                
                # --- Validações de Entrada ---
                if not (0 <= row <= 8 and 0 <= col <= 8 and 0 <= value <= 9): # Permite 0 para remover número
                    print("Coordenadas e/ou valor fora do intervalo (1-9) para números. 0 remove.")
                    continue
                
                # Verifica se a célula é editável (número inicial)
                if not self.editable_mask[row, col]:
                    print("Esta célula é um número inicial (verde) e não pode ser alterada.")
                    continue
				
                # --- Aplica o Movimento ---
                if value != self.solution_matrix[row, col]:
                     
                     self.erros += 1
                     print(f"Peça {value} incorreta. Erro {self.erros}/{self.max_err}")
                else:
                     self.matrix[row, col] = value
                
                # --- Verifica a Vitória ---
                # Verifica se o tabuleiro preenchido (sem zeros) é igual à solução
                if (0 not in self.matrix) and np.array_equal(self.matrix, self.solution_matrix):
                    self.end = True
                    self.show_matrix()
                    print("\n🎉 Parabéns! Você completou o Sudoku! 🎉")
                elif 0 not in self.matrix:
                     # Se estiver totalmente preenchido mas incorreto
                     print("O tabuleiro está preenchido, mas não está correto. Continue tentando!")

            except ValueError:
                print("Entrada inválida. Certifique-se de que linha, coluna e valor são números.")
            except Exception as e:
                print(f"Ocorreu um erro inesperado: {e}")

# Início do Jogo
partida = Sudoku()
partida.run()

=== Bem-vindo ao jogo de Sudoku! ===

  1 2 3   4 5 6   7 8 9
-------------------------
| 7 4 3 | 5 . 9 | . 8 2 | 1
| 5 . . | . . 8 | . . 3 | 2
| 1 . 6 | . . 7 | . 9 5 | 3
-------------------------
| . . 5 | 9 3 . | 2 . 7 | 4
| 9 7 2 | 1 . . | 3 5 6 | 5
| . . 4 | . 7 5 | 9 1 . | 6
-------------------------
| . . . | . . . | . 2 9 | 7
| 4 . . | . 9 2 | 8 . 1 | 8
| . . . | . 5 3 | 7 . 4 | 9
-------------------------

Para jogar, insira (linha coluna valor), ex: 1 2 5
Peça 3 incorreta. Erro 1/10

  1 2 3   4 5 6   7 8 9
-------------------------
| 7 4 3 | 5 . 9 | . 8 2 | 1
| 5 . . | . . 8 | . . 3 | 2
| 1 . 6 | . . 7 | . 9 5 | 3
-------------------------
| . . 5 | 9 3 . | 2 . 7 | 4
| 9 7 2 | 1 . . | 3 5 6 | 5
| . . 4 | . 7 5 | 9 1 . | 6
-------------------------
| . . . | . . . | . 2 9 | 7
| 4 . . | . 9 2 | 8 . 1 | 8
| . . . | . 5 3 | 7 . 4 | 9
-------------------------

Para jogar, insira (linha coluna valor), ex: 1 2 5
Peça 3 incorreta. Erro 2/10

  1 2 3   4 5 6   7 8 9
----------------